In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
week14_link = 'http://www.footballlocks.com/nfl_odds_week_14.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week14_html = requests.get(week14_link).text

soup = BeautifulSoup(week14_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[277])

<tr>
<td>12/7 8:25 ET</td>
<td>At Atlanta</td>
<td>-2.5</td>
<td>New Orleans</td>
<td>51.5</td>
<td>-$140 +$120</td>
</tr>
<tr>
<td>12/11 8:30 ET</td>
<td>Chicago</td>
<td>-6</td>
<td>At St. Louis</td>
<td>41.5</td>
<td>-$275 +$225</td>
</tr>


In [3]:
#want to 278 because it does not include last one
table = tr[51:278]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week14_df = pd.DataFrame(string_table)

week14_df.columns = ['all_data']

week14_df.head()

,all_data
0,<tr>\n<td>12/7 8:25 ET</td>\n<td>At Atlanta</t...
1,<tr>\n<td>12/10 1:00 ET</td>\n<td>At Tampa Bay...
2,<tr>\n<td>12/10 1:00 ET</td>\n<td>At Cincinnat...
3,<tr>\n<td>12/10 1:00 ET</td>\n<td>At Buffalo</...
4,<tr>\n<td>12/10 4:25 ET</td>\n<td>At Jacksonvi...


In [4]:
#want to clean the date data

dates = [x.split('td')[1] for x in week14_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week14_df['date'] = cleaned_dates

In [5]:
#pull out favorites

week14_df['cleaned_favorites'] = week14_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week14_df['cleaned_underdogs'] = week14_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week14_df['cleaned_spread'] = week14_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week14_df['cleaned_over_under'] = week14_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week14_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week14_df['money_odds'] = money_odds

#input week number for each game
week14_df['week_number'] = '14'

In [6]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2017
year_list = []
for x in week14_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week14_df['year'] = year_list

In [7]:
#create a final dataframe for week 14
week14_final = week14_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week14_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,12/7 8:25 ET,2017,14,At Atlanta,New Orleans,-2.5,51.5,-$140 +$120
1,12/10 1:00 ET,2017,14,At Tampa Bay,Detroit,"span title=""Pick Em. No Favorite. No Underdog...",48,-$110 -$110
2,12/10 1:00 ET,2017,14,At Cincinnati,Chicago,-6,40,-$265 +$225
3,12/10 1:00 ET,2017,14,At Buffalo,Indianapolis,-3,36.5,-$180 +$160
4,12/10 4:25 ET,2017,14,At Jacksonville,Seattle,-2.5,41,-$145 +$125


In [8]:
#week14_final to pickle
week14_final.to_pickle('week14_final.pkl')